In [1]:
PROJECT_ID = 'razekmh-angola'

In [2]:
# install libraries
! pip install -q earthengine-api geemap

In [3]:
# import libraries
import ee
import geemap

In [4]:
# authenticate to Earth Engine
ee.Authenticate()

True

In [5]:
# initialize the project    
ee.Initialize(project=PROJECT_ID)

In [6]:
# make a map
Map = geemap.Map(center=[-12.0, 18.0], zoom=8)

In [7]:
# add outline of Angola to the map

countries = ee.FeatureCollection('FAO/GAUL/2015/level0')
angola_outline = countries.filter(ee.Filter.eq('ADM0_NAME', 'Angola'))
Map.addLayer(angola_outline, {}, 'Angola')

In [8]:
landsat_8_image_collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
    .filterBounds(angola_outline) \
        .filterDate('2023-01-01', '2024-01-01') \
            .filterMetadata('CLOUD_COVER', 'less_than', 25)

# Applies scaling factors. ref # https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2#colab-python
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

landsat_8_image_collection = landsat_8_image_collection.map(apply_scale_factors)

visualization = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
}

In [9]:
# display the map
Map.addLayer(landsat_8_image_collection, visualization, 'Landsat 8')
Map

Map(center=[-12.0, 18.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…